# Real Bookshelf to Virtual Bookshelf 📚

This application takes an image of a real life bookshelf and generates a digital list of books, along with a Google Books link for each of them.

[GitHub Repo](https://github.com/karangt/bookshelf)


**Usage**
- Please wait for the app to load.
- Select or upload an image and then click on the 'Process Image' button below.  
- The uploaded image should be similar to the demo images i.e. of a single bookshelf and without tilted books


Karan Gupta 

***

In [26]:
# Run this app using the following command:
#
# voila app.ipynb

In [27]:
import cv2
import ipywidgets as widgets
import numpy as np


%load_ext autoreload
%autoreload 1
%aimport utils
%aimport segment
%aimport gocr
%aimport east

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Load the initial files

startImagePath = "images/bookshelf3a.jpeg"

# For processing
#IMG = cv2.imread(startImagePath)

# For displaying
file = open(startImagePath,"rb")
startImage = file.read()

# Placeholder image
file2 = open("images/placeholder.jpg","rb")
phImage = file2.read()



In [29]:
def on_rbutton_change(change):
    '''
    Triggered when radio button state is changed
    '''
    
    selection = change["new"]
    
    mapping = {"Demo 1":"images/bookshelf3a.jpeg",
               "Demo 2":"images/bookshelf3b.jpeg",
               "Demo 3":"images/Good-Book-Spines.jpg"}
    
    # Reset the output and segOutput widgets
    output.clear_output()
    bookInfoOutput.value =''
    display(bookInfoOutput)
    
    segOutput.value = phImage
    display(segOutput)
    
    # Update status widget
    status.clear_output()
    with status: print("Updating image..")
        
    if selection in ["Demo 1","Demo 2","Demo 3"]:
        # Disable the uploader
        uploader.disabled = True
        display(uploader)
        
        # Display the image
        file = open(mapping[selection],"rb")
        tempImage = file.read()
        imgDisplay.value = tempImage
        display(imgDisplay)
        # Enable processing button
        button.disabled = False
        display(button)
        
        status.clear_output()
        with status: print("Image updated! Click 'Process Image'")

    else: # upload
        # Enable the uploader
        uploader.disabled = False
        display(uploader)
        # Disable processing button
        button.disabled = True
        display(button)
        # Remove image
        imgDisplay.value=phImage
        display(imgDisplay)
        
        status.clear_output()
        with status: print("Select an image and wait for the 'Image Uploaded' message.")


In [30]:
# Set up uploader events
def on_upload(change):
    '''
    Triggered when image is uploaded
    '''
    for v in uploader.value.values():
        
        imgDisplay.value=v["content"]
        display(imgDisplay)
        # Enable processing button
        button.disabled = False
        display(button)
        
        status.clear_output()
        with status: print("Image uploaded! Click 'Process Image'")
        break

In [31]:
def getHTMLTable(booksInfo):
    # CSS style generated using https://divtable.com/table-styler/
    html='''
    <style type="text/css">
        table.minimalistBlack {
          border: 1px solid #000000;
          width: 100%;
          text-align: left;
          border-collapse: collapse;
        }
        table.minimalistBlack td, table.minimalistBlack th {
          border: 1px solid #000000;
          padding: 4px 4px;
        }
        table.minimalistBlack tbody td {
          font-size: 13px;
        }
        table.minimalistBlack thead {
          background: #CFCFCF;
          background: -moz-linear-gradient(top, #dbdbdb 0%, #d3d3d3 66%, #CFCFCF 100%);
          background: -webkit-linear-gradient(top, #dbdbdb 0%, #d3d3d3 66%, #CFCFCF 100%);
          background: linear-gradient(to bottom, #dbdbdb 0%, #d3d3d3 66%, #CFCFCF 100%);
          border-bottom: 2px solid #000000;
        }
        table.minimalistBlack thead th {
          font-size: 15px;
          font-weight: bold;
          color: #000000;
          text-align: left;
        }
        table.minimalistBlack tfoot td {
          font-size: 14px;
        }
    </style>
    
    <table class="minimalistBlack">
        <tr>
            <th>SN</th>
            <th>Title</th>
            <th>Author</th>
            <th>Google Link</th>
            <th>GoodReads Link</th>
            <th>OCR Text</th>
        </tr>
        '''
    i=1
    for b in booksInfo:
        html += '''
        <tr>
            <td>{}</td>
            <td>{}</td>
            <td>{}</td>
            <td><a href="{}" target="_blank">Link</a></td>
            <td><a href="{}" target="_blank">Link</a><br/></td>
            <td>{}</td>
        </tr>
        '''.format(i,b["title"],b["author"],b["google_link"],b["goodreads_link"],b["query"])
        i+=1
    html+="</table>"
    return html

In [32]:

def processImage(img,path):
    im_eq = utils.imgPreprocess(img)
    with output: print("Detecting text ...")
    frame, framev, boxes, indices = east.detectText(im_eq,0.5,0.3)
    with output: print("Segmenting books ...")
    sboxes = segment.getSortedBoxes(boxes,indices,0.1)
    sboxes = segment.trimBoxes(frame,sboxes)
    bookList = segment.boxesToBooks(sboxes, debug=True)
    bookImages = segment.getBookImages(frame,bookList,padY=frame.shape[0])
    with output: print("Displaying segmented books ...")
    segment.showSegmentedBooks(bookImages,path)
   
    file = open(path,"rb")
    dispImage = file.read()
    segOutput.value = dispImage
    display(segOutput)
    
    with output: print("Getting book links (please wait) ...")
    
    booksInfo = gocr.getAllBookLinks(bookImages)
    
    bookInfoOutput.value = getHTMLTable(booksInfo)
    display(bookInfoOutput)
    return

In [33]:
# getHTMLTable([{"title":"test1","google_link":"http://test.com","goodreads_link":"http://goodreads.com"},
#               {"title":"test2","google_link":"http://test.com","goodreads_link":"http://goodreads.com"}
#              ])

In [34]:
def on_button_clicked(change):
    button.disabled = True
    display(button)
    
    img = cv2.imdecode(np.frombuffer(imgDisplay.value, np.uint8), -1)
    processImage(img,"images/tmp/seg_output.png")
    
    button.disabled = False
    display(button)
    
    with output: print("✅ DONE! Scroll down ⬇")

## Input

In [39]:
# Setup and display all the widgets
uploader = widgets.FileUpload( accept='image/*', multiple=False, disabled=True)
imgDisplay = widgets.Image(value= startImage, width=400)
rbutton = widgets.RadioButtons(
    options=['Demo 1','Demo 2','Demo 3', 'Upload'],
    description='Image',
    disabled=False,
    layout={'width': 'max-content'}
)
button = widgets.Button(
    description='Process Image',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Process the image to create a list of books',
)
status = widgets.Output()
output = widgets.Output()
segOutput = widgets.Image(width=800,value=phImage)
bookInfoOutput = widgets.HTML()
loadingGif = widgets.Image(width=10)


display(rbutton,status,uploader,imgDisplay)

rbutton.observe(on_rbutton_change,"value")
button.on_click(on_button_clicked)
uploader.observe(on_upload, names='value')

RadioButtons(description='Image', layout=Layout(width='max-content'), options=('Demo 1', 'Demo 2', 'Demo 3', '…

Output()

FileUpload(value={}, accept='image/*', description='Upload', disabled=True)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00`\x00`\x00\x00\xff\xe1\x00\x8cExif\x00\x00MM\x00…

In [41]:
display(button)

Button(button_style='info', description='Process Image', style=ButtonStyle(), tooltip='Process the image to cr…

***
## Processing Results

In [37]:
display(output,segOutput,bookInfoOutput)

Output()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00`\x00`\x00\x00\xff\xe1\x00\x8cExif\x00\x00MM\x00…

HTML(value='')